In [ ]:
class Hedger:
    """Class for hedging simulation."""

    def __init__(self, pricer: BlackScholesPricer, paths: np.ndarray, N: int, T: float):
        self.pricer = pricer
        self.greeks = Greeks(pricer)
        self.paths = paths
        self.N = N
        self.T = T

    def hedge(self, option_type="call"):
        """Perform delta hedging simulation."""
        pnl = np.zeros((self.N, self.paths.shape[1]))
        dt = self.T / self.N
        temp_pricer = self.pricer.copy()

        for i in range(self.paths.shape[1]):
            S0 = self.paths[0, i]
            tau = self.T
            temp_pricer.S0 = S0
            delta_old = self.greeks.delta(S0, tau, option_type)
            option_value_old = (
                self.pricer.price_call() if option_type == "call" else self.pricer.price_put()
            )
            cash_position = option_value_old - delta_old * S0
            portfolio_value = delta_old * S0 + cash_position

            for j in range(1, self.N):
                St = self.paths[j, i]
                tau = self.T - j * dt
                temp_pricer.S0 = St
                delta_new = self.greeks.delta(St, tau, option_type)
                delta_diff = delta_new - delta_old
                cash_position *= np.exp(self.pricer.r * dt)
                cash_position -= delta_diff * St
                portfolio_value = delta_new * St + cash_position
                option_value = (
                    self.pricer.price_call() if option_type == "call" else self.pricer.price_put()
                )
                pnl[j, i] = portfolio_value - option_value
                delta_old = delta_new

        return pnl